In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os

In [2]:
input_shape = (24, 128, 128, 3)
# Define the encoder
def create_unet(input_shape):
    input = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(input)
    #x = tf.keras.layers.MaxPooling3D((2, 2, 2), padding='same')(x)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         +x = tf.keras.layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(x)
    #x = tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
    #x = tf.keras.layers.UpSampling3D((2, 2, 2))(x)
    output = tf.keras.layers.Conv3D(3, (3, 3, 3), activation='sigmoid', padding='same')(x)
    
    model = tf.keras.Model(input, output)
    return model
model=create_unet(input_shape)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24, 128, 128,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 24, 128, 128,   │         5,248 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 24, 128, 128,   │         5,187 │
│                                 │ 3)                     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,435 (40.76 KB)

 Trainable params: 10,435 (40.76 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
model.compile(optimizer='adam', loss='mse')

In [4]:
# Function to load and preprocess videos
def load_videos(folder, frame_size, frame_count):
    frames = []
    for filename in os.listdir(folder):
        if filename.endswith('.mp4'):
            cap = cv2.VideoCapture(os.path.join(folder, filename))
            video_frames = []
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, frame_size)
                video_frames.append(frame)
                if len(video_frames) == frame_count:
                    frames.append(video_frames)
                    video_frames = []
            cap.release()
    frames = np.array(frames)
    frames = frames / 255.0  # Normalize
    return frames

# Load and preprocess videos
frame_size = (128, 128)
frame_count = 24
video_frames_in = load_videos('dataset/1', frame_size, frame_count)
video_frames_out = load_videos('dataset/0', frame_size, frame_count)

In [5]:
min_samples = min(len(video_frames_in), len(video_frames_out))
video_frames_in = video_frames_in[:min_samples]
video_frames_out = video_frames_out[:min_samples]


In [6]:
# Train the autoencoder
model.fit(video_frames_in, video_frames_out, epochs=10, batch_size=1)


MemoryError: Unable to allocate 6.22 GiB for an array with shape (1416, 24, 128, 128, 3) and data type float32

In [ ]:
model.save('mygg.h5')

In [ ]:
'''# Function to save video
def save_video(filename, frames, frame_size):
    height, width = frame_size
    video = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))
    for frame_set in frames:
        for frame in frame_set:
            frame = (frame * 255).astype(np.uint8)
            video.write(frame)
    video.release()

# Save reconstructed video
save_video('reconstructed_cat2.mp4', video_frames, frame_size)

print("Video processing complete.")'''

'# Function to save video\ndef save_video(filename, frames, frame_size):\n    height, width = frame_size\n    video = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*\'mp4v\'), 30, (width, height))\n    for frame_set in frames:\n        for frame in frame_set:\n            frame = (frame * 255).astype(np.uint8)\n            video.write(frame)\n    video.release()\n\n# Save reconstructed video\nsave_video(\'reconstructed_cat2.mp4\', video_frames, frame_size)\n\nprint("Video processing complete.")'